# **Modelado Avanzado - `06_modelado_avanzado.ipynb`**

### 🎯 **Objetivo del Notebook**

Este notebook tiene como objetivo aplicar y evaluar modelos de clasificación más avanzados sobre las características extraídas de las señales EEG, con el fin de mejorar la detección de eventos motores (*Grasp-and-Lift*) a partir de señales cerebrales. Partimos de los datos ya preprocesados en el notebook anterior (características temporales + frecuencia, con outliers eliminados y normalización).

---

### 📌 **Contexto**

En el notebook anterior (`06_ingenieria_caracteristicas.ipynb`), extraímos nuevas variables a partir de ventanas temporales de la señal EEG, combinando estadísticas en el dominio temporal y energía por bandas en el dominio de la frecuencia (potencia espectral por canal). Estas nuevas variables, al ser introducidas en un modelo base (`LogisticRegression`), generaron una mejora notable en el rendimiento, alcanzando AUC-ROC superiores a 0.94 en varios eventos.

Dado el éxito de estas nuevas representaciones, en este notebook las utilizaremos para probar modelos más potentes y complejos, que podrían explotar aún mejor la información contenida en las señales.

---

### 🤖 **Análisis de Modelos Avanzados a Utilizar**

Por tanto, tras aplicar ingeniería de características sobre las señales EEG, disponemos ahora de un conjunto de datos **tabular y numérico**, que incluye estadísticas temporales y características en frecuencia por canal. Esto nos permite aplicar modelos de clasificación avanzados que trabajen bien con este tipo de representación.

A continuación, se resumen los modelos más adecuados para este escenario, con sus ventajas y consideraciones:


#### 🔲 Random Forest (RF)
- ✅ Robusto frente a ruido y datos no lineales.
- ✅ No requiere mucho ajuste.
- ✅ Interpretable (permite ver la importancia de las características).
- 🔸 Puede ser una excelente línea base avanzada.
  
> **Recomendado como primer modelo avanzado.**

#### 🟠 XGBoost
- 📈 Muy potente y usado en competiciones de ciencia de datos.
- ✅ Excelente capacidad para capturar relaciones complejas.
- ✅ Controla bien el sobreajuste.
- 🔸 Requiere ajuste fino de hiperparámetros para rendir al máximo.

> **Recomendado como modelo principal si se desea máxima precisión.**

#### 🟢 LightGBM
- ⚡ Muy rápido y eficiente, incluso con muchos datos.
- ✅ Similar a XGBoost pero con menor consumo de recursos.
- 🔸 Puede requerir ajustes para manejar datos desbalanceados.

> **Ideal si se necesita alta velocidad de entrenamiento.**

#### 🔵 Redes Neuronales (MLP / Keras)
- ✅ Capaces de aprender representaciones complejas.
- 🔸 Menos interpretables.
- 🔸 Requieren más datos y ajuste fino para evitar sobreajuste.

> **Opción interesante si se quiere explorar arquitectura neuronal simple sobre datos tabulares.**

#### ⚫ SVM (Support Vector Machines)
- ❌ Poca escalabilidad con muchos datos.
- ❌ No recomendable para clasificación multiclase o multietiqueta con gran volumen de muestras.

> **No se recomienda como opción principal para este caso.**
> 

#### 🧠 **Conclusión**

Dado el tipo de datos actual (ventanas EEG convertidas a características numéricas tabulares), se considera que los modelos más apropiados para probar en este notebook son:

1. `RandomForestClassifier`
2. `XGBoostClassifier`
3. `LightGBMClassifier`
4. (Opcional) `MLPClassifier` o redes con Keras

Estos modelos serán evaluados por separado para cada evento motor, comparando su rendimiento con el modelo base (`LogisticRegression`), y seleccionando los más prometedores para integrar en el pipeline final.

---

### **🚀 Flujo de Trabajo en este Notebook**

1️⃣ Carga de los datos preprocesados (características extraídas + normalización)

2️⃣ Entrenamiento de modelos avanzados:
- **`RandomForestClassifier`**
- **`XGBoostClassifier`**
- **`LightGBMClassifier`**

3️⃣ Evaluación de rendimiento por evento utilizando métricas como:
- AUC-ROC
- (Opcional) F1-score, accuracy

4️⃣ Comparación de resultados frente al modelo base (`LogisticRegression`)

5️⃣ **Selección del mejor modelo** y guardado para uso posterior

---

### **💡 Objetivo final**
Identificar el modelo con mejor capacidad de generalización para cada evento motor, y dejarlo listo para usar en el pipeline de predicción final.


## **1. Carga de Datos Preprocesados**

In [2]:
import os
import pickle
import pandas as pd

# Ruta de los datos procesados
processed_path = r"C:\Users\luciaft\Documents\TFG\TFG\graspAndLiftDetectionTFGProyect\data\processed"
data_file = os.path.join(processed_path, "preprocessed_features_temporal_freq.pkl")

# Cargar los datos preprocesados
with open(data_file, "rb") as f:
    X_train_scaled, y_train_win, X_valid_scaled, y_valid_win = pickle.load(f)

# Verificar formas
print("✅ Datos cargados correctamente:")
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"y_train_win shape: {y_train_win.shape}")
print(f"X_valid_scaled shape: {X_valid_scaled.shape}")
print(f"y_valid_win shape: {y_valid_win.shape}")

✅ Datos cargados correctamente:
X_train_scaled shape: (16299, 512)
y_train_win shape: (16299, 6)
X_valid_scaled shape: (3700, 512)
y_valid_win shape: (3700, 6)


## **2. Entrenamiento de Modelos Avanzados**